In [1]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 8, 2021...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


\--- Day 8: Seven Segment Search ---
----------

You barely reach the safety of the cave when the whale smashes into the cave mouth, collapsing it. Sensors indicate another exit to this cave at a much greater depth, so you have no choice but to press on.

As your submarine slowly makes its way through the cave system, you notice that the four-digit [seven-segment displays](https://en.wikipedia.org/wiki/Seven-segment_display) in your submarine are malfunctioning; they must have been damaged during the escape. You'll be in a lot of trouble without them, so you'd better figure out what's wrong.

Each digit of a seven-segment display is rendered by turning on or off any of seven segments named `a` through `g`:

```
  0:      1:      2:      3:      4:
 aaaa    ....    aaaa    aaaa    ....
b    c  .    c  .    c  .    c  b    c
b    c  .    c  .    c  .    c  b    c
 ....    ....    dddd    dddd    dddd
e    f  .    f  e    .  .    f  .    f
e    f  .    f  e    .  .    f  .    f
 gggg    ....    gggg    gggg    ....

  5:      6:      7:      8:      9:
 aaaa    aaaa    aaaa    aaaa    aaaa
b    .  b    .  .    c  b    c  b    c
b    .  b    .  .    c  b    c  b    c
 dddd    dddd    ....    dddd    dddd
.    f  e    f  .    f  e    f  .    f
.    f  e    f  .    f  e    f  .    f
 gggg    gggg    ....    gggg    gggg

```

So, to render a `1`, only segments `c` and `f` would be turned on; the rest would be off. To render a `7`, only segments `a`, `c`, and `f` would be turned on.

The problem is that the signals which control the segments have been mixed up on each display. The submarine is still trying to display numbers by producing output on signal wires `a` through `g`, but those wires are connected to segments *randomly*. Worse, the wire/segment connections are mixed up separately for each four-digit display! (All of the digits *within* a display use the same connections, though.)

So, you might know that only signal wires `b` and `g` are turned on, but that doesn't mean *segments* `b` and `g` are turned on: the only digit that uses two segments is `1`, so it must mean segments `c` and `f` are meant to be on. With just that information, you still can't tell which wire (`b`/`g`) goes to which segment (`c`/`f`). For that, you'll need to collect more information.

For each display, you watch the changing signals for a while, make a note of *all ten unique signal patterns* you see, and then write down a single *four digit output value* (your puzzle input). Using the signal patterns, you should be able to work out which pattern corresponds to which digit.

For example, here is what you might see in a single entry in your notes:

```
acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf
```

(The entry is wrapped here to two lines so it fits; in your notes, it will all be on a single line.)

Each entry consists of ten *unique signal patterns*, a `|` delimiter, and finally the *four digit output value*. Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are). The unique signal patterns correspond to the ten different ways the submarine tries to render a digit using the current wire/segment connections. Because `7` is the only digit that uses three segments, `dab` in the above example means that to render a `7`, signal lines `d`, `a`, and `b` are on. Because `4` is the only digit that uses four segments, `eafb` means that to render a `4`, signal lines `e`, `a`, `f`, and `b` are on.

Using this information, you should be able to work out which combination of signal wires corresponds to each of the ten digits. Then, you can decode the four digit output value. Unfortunately, in the above example, all of the digits in the output value (`cdfeb fcadb cdfeb cdbaf`) use five segments and are more difficult to deduce.

For now, *focus on the easy digits*. Consider this larger example:

```
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb |
fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec |
fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef |
cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega |
efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga |
gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf |
gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf |
cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd |
ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg |
gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc |
fgae cfgab fg bagce

```

Because the digits `1`, `4`, `7`, and `8` each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Counting *only digits in the output values* (the part after `|` on each line), in the above example, there are `*26*` instances of digits that use a unique number of segments (highlighted above).

*In the output values, how many times do digits `1`, `4`, `7`, or `8` appear?*

In [2]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    entries = [x.strip() for x in f.readlines()]

# Verify parse
pprint(entries[:6])

['gbcefa eac acfbg ae dcabfg begcdaf ecgba fgaedc beaf gcbde | cbgfa gedcb '
 'fgecab fbagdc',
 'fb adceb febagd edgafcb cbfega gecadf dgfb dfeag baefd baf | gbdf gbfd '
 'fgebac dfega',
 'dgabec cfgeb cfb cefd cdabfg gbdce fc begdfc gfbea febacdg | bcfdaeg cf fc '
 'gecabdf',
 'gadfce acdgf bgaec gbfdca acgbd dceagfb agfdbe dbfc gbd bd | dgb cbadg cagdb '
 'cfbd',
 'edcbafg ab ecfdb dacfeg bcag aeb fgaebd bfecga eacgf bcaef | ba eab cdgefa '
 'gcab',
 'gbdcfea adfcge ebc be fegcbd ecfbad efgcd ebdg gacfb fcebg | cbgefd defcg '
 'cbefdg eafgcdb']


In [3]:
sseg_by_count = {
    "1": 2,
    "2": 5,
    "3": 5,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 3,
    "8": 7,
    "9": 6,
    "0": 6,
}
unique_len = {k: v for k,v in sseg_by_count.items() if list(sseg_by_count.values()).count(v) == 1}
print("unique:", unique_len)


unique: {'1': 2, '4': 4, '7': 3, '8': 7}


In [4]:
count = 0
for entry in entries:
    input, output = entry.split(' | ')
    digits = output.split()
    for digit in digits:
        wires = list(digit)
        if len(wires) in unique_len.values():
            count += 1

answer1 = count

In [5]:
print("answer 1:", answer1)

answer 1: 412


----

In [6]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 8, 2021...
Saving puzzle description to "README.md"...
Done!


\--- Part Two ---
----------

Through a little deduction, you should now be able to determine the remaining digits. Consider again the first example above:

```
acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab |
cdfeb fcadb cdfeb cdbaf
```

After some careful analysis, the mapping between signal wires and segments only make sense in the following configuration:

```
 dddd
e    a
e    a
 ffff
g    b
g    b
 cccc

```

So, the unique signal patterns would correspond to the following digits:

* `acedgfb`: `8`
* `cdfbe`: `5`
* `gcdfa`: `2`
* `fbcad`: `3`
* `dab`: `7`
* `cefabd`: `9`
* `cdfgeb`: `6`
* `eafb`: `4`
* `cagedb`: `0`
* `ab`: `1`

Then, the four digits of the output value can be decoded:

* `cdfeb`: `*5*`
* `fcadb`: `*3*`
* `cdfeb`: `*5*`
* `cdbaf`: `*3*`

Therefore, the output value for this entry is `*5353*`.

Following this same process for each entry in the second, larger example above, the output value of each entry can be determined:

* `fdgacbe cefdb cefbgd gcbe`: `8394`
* `fcgedb cgb dgebacf gc`: `9781`
* `cg cg fdcagb cbg`: `1197`
* `efabcd cedba gadfec cb`: `9361`
* `gecf egdcabf bgf bfgea`: `4873`
* `gebdcfa ecba ca fadegcb`: `8418`
* `cefg dcbef fcge gbcadfe`: `4548`
* `ed bcgafe cdgba cbgef`: `1625`
* `gbdfcae bgc cg cgb`: `8717`
* `fgae cfgab fg bagce`: `4315`

Adding all of the output values in this larger example produces `*61229*`.

For each entry, determine all of the wire/segment connections and decode the four-digit output values. *What do you get if you add up all of the output values?*

In [7]:
import itertools

# Expected wiring
digit_wires = {
    "abcefg":   "0",
    "cf":       "1",
    "acdeg":    "2",
    "acdfg":    "3",
    "bcdf":     "4",
    "abdfg":    "5",
    "abdefg":   "6",
    "acf":      "7",
    "abcdefg":  "8",
    "abcdfg":   "9",
}

def translate(digits, mapping, verbose=False):
    """Apply a wiring map to a signal."""
    if verbose: print("digits:", digits, "mapping:", mapping)
    mapped = [list(x) for x in digits]
    for x in range(len(mapped)):
        for y in range(len(mapped[x])):
            mapped[x][y] = mapping[digits[x][y]]

    return [''.join(sorted(x)) for x in mapped]

def brute_force(digits, target=digit_wires.keys(), verbose=False):
    """Finds a wiring map which results in 10 valid digits."""
    sseg = [chr(x) for x in range(ord('a'), ord('g')+1)]
    # Iterate through all wiring combinations
    for a,b,c,d,e,f,g in itertools.product(sseg, sseg, sseg, sseg, sseg, sseg, sseg):
        # Early exit if duplicate wiring
        if len(set([a,b,c,d,e,f,g])) < 7: continue
        mapping = {'a': a, 'b': b, 'c': c, 'd': d, 'e': e, 'f': f, 'g': g}
        # Re-wire the input with the mapping
        mapped = translate(digits, mapping)
        # Test the wiring against expected
        if all([d in target for d in mapped]):
            return mapping
    assert False
    
def decode_digits(digits):
    """Lookup active wires against a decimal value."""
    return [digit_wires[d] for d in digits]

# Validate solution against example
mapping = brute_force("acedgfb cdfbe gcdfa fbcad dab cefabd cdfgeb eafb cagedb ab".split(), verbose=False)
digits = translate("cdfeb fcadb cdfeb cdbaf".split(), mapping, verbose=False)
assert decode_digits(digits) == ['5', '3', '5', '3']

In [8]:
answer2 = 0
for entry in entries:
    # Split input|output
    input, output = entry.split(' | ')

    # Find wiring map that results in 10 valid digits
    mapping = brute_force(input.split())
    # Convert output with wiring map
    output = translate(output.split(), mapping)
    # Decode wires to decimal
    digits = decode_digits(output)

    answer2 += int("".join(digits))

In [9]:
print("answer2", answer2)

answer2 978171
